# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [1]:
%%capture
!make local

In [2]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.calc import aggregateResults
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import dropNonSkydiveDataFrom
from ssscoring.calc import getSpeedSkydiveFrom
from ssscoring.calc import isValidJump
from ssscoring.calc import isValidMinimumAltitude
from ssscoring.calc import jumpAnalysisTable
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import roundedAggregateResults
from ssscoring.calc import totalResultsFrom
from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.flysight import getAllSpeedJumpFilesFrom
from ssscoring.flysight import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [3]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Paraclete XP| 304 |
| SkyDance SkyDiving | 100 |
| Skydive Arizona | 1509 |
| Skydive Chicago | 616 |
| Thai Sky Adventures | 21 |

In [4]:
dropZoneAltMSL = 616
ignoreBaseline = True # Set to False to evaluate any baseline tracks present in the data lake.

In [5]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green"> 616.00 ft</span> ( 187.76 m)<h1>')

In [6]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [7]:
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

/opt/conda/lib/python3.11/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


### Air density

**<a href='https://aviex.goflexair.com/blog/density-of-air-air-density' target='_blank'>Pilot's Guide to Air Density</a>** - A useful guide with calculations, lots of information.

![](https://ciurana.eu/personal/air-density-partial-table.png)

---
## Results

In [8]:
aggregate = aggregateResults(jumpResults)
if ignoreBaseline:
    aggregate = aggregate[~aggregate.index.str.contains('baseline')]
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
Miller-2024-championship-official R1_12-43-43:v1,316.1295,127.152,201.240,262.584,299.448,316.944,25.0,317.628
Miller-2024-championship-official R2_14-27-13:v1,333.9135,132.912,228.744,285.768,325.116,334.908,25.0,335.988
Miller-2024-championship-official R3_15-53-00:v1,327.07575,152.568,256.284,312.336,326.520,306.720,25.0,328.104
Miller-2024-championship-official R4_17-24-24:v1,331.31925,153.540,252.864,307.836,325.548,329.436,25.0,332.460
Miller-2024-championship-official R5_18-49-36:v1,348.2505,153.684,254.232,311.184,331.848,346.500,25.0,349.092
Miller-2024-championship-official R6_12-49-38:v1,326.80125,143.820,223.740,289.440,323.676,242.316,25.0,333.000
Miller-2024-championship-official R7_14-20-26:v1,324.4995,134.964,211.212,262.332,285.228,318.744,25.0,327.816
Miller-2024-championship-official R8_16-05-15:v1,329.84325,147.816,244.008,300.636,318.456,317.988,25.0,332.532


### Rounded results for training log

In [9]:
roundedResults = roundedAggregateResults(aggregate)

### All jumps

In [10]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [11]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [12]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    if ignoreBaseline and 'baseline' in resultRef:
        continue
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">Miller-2024-championship-official R1_12-43-43:v1 jump - 🟢 valid …

HTML(value='<h3>Max speed = 318; exit at 4171 m (13685 ft), end scoring window at 1915 m (6284 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,127,97,52.6,163.1,13305,160
,10.0,201,99,63.9,287.0,12552,224
,15.0,263,119,65.6,423.1,11486,288
,20.0,299,125,67.4,578.2,10193,324
,25.0,317,119,69.4,734.6,8780,339


HTML(value='<hr><h1><span style="color: #0f0">Miller-2024-championship-official R2_14-27-13:v1 jump - 🟢 valid …

...

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,135,132,45.6,207.9,13071,189
,10.0,211,124,59.6,384.2,12280,245
,15.0,262,134,62.9,562.2,11190,295
,20.0,285,136,64.6,753.9,9954,316
,25.0,319,124,68.8,928.9,8569,342


HTML(value='<hr><h1><span style="color: #0f0">Miller-2024-championship-official R8_16-05-15:v1 jump - 🟢 valid …

HTML(value='<h3>Max speed = 333; exit at 4241 m (13914 ft), end scoring window at 1985 m (6512 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,148,141,46.4,221.0,13510,204
,10.0,244,124,63.0,403.9,12607,274
,15.0,301,118,68.5,574.5,11369,323
,20.0,318,121,69.2,737.3,9943,341
,25.0,318,109,71.1,892.4,8471,336


---
## Summary of all speed skydives in the data lake

**<a href='https://intimescoring.com/Results/EventResults?dbid=100&mid=1074&smid=0&mdcid=1273&dseid=0' target='_blank'>InTime Scoring - 2024 USPA Speed Open results</a>**

In [13]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
Miller-2024-championship-official R1_12-43-43:v1,316,127,201,263,299,317,25.0,318
Miller-2024-championship-official R2_14-27-13:v1,334,133,229,286,325,335,25.0,336
Miller-2024-championship-official R3_15-53-00:v1,327,153,256,312,327,307,25.0,328
Miller-2024-championship-official R4_17-24-24:v1,331,154,253,308,326,329,25.0,332
Miller-2024-championship-official R5_18-49-36:v1,348,154,254,311,332,346,25.0,349
Miller-2024-championship-official R6_12-49-38:v1,327,144,224,289,324,242,25.0,333
Miller-2024-championship-official R7_14-20-26:v1,324,135,211,262,285,319,25.0,328
Miller-2024-championship-official R8_16-05-15:v1,330,148,244,301,318,318,25.0,333


,totalSpeed,meanSpeed,maxScore
totalSpeed,2637.8325,329.729062,348.2505
